In [ ]:
##### Purpose:  
##### Author: Julia Cope
##### Creation Date: 4/20/23
##### Project: A2 NLP - capturing climate claims 
##### Inputs: 
##### Inputs: 
##### Inputs: 
##### Output: 
##### Output: 
##### Output: 

In [1]:
import os
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA


In [2]:
pd.options.display.max_colwidth = 150

In [3]:
def func_make_evidence_df(df, ID_column, column_name):
    
    evidence_df = df[[ID_column, column_name]].explode(column_name)
    temp_df = pd.json_normalize(evidence_df[column_name])
    temp_df.reset_index(inplace=True, drop=True)
    evidence_df.reset_index(inplace=True, drop=True)
    evidence_df = pd.concat([evidence_df['claim_id'], temp_df], axis=1)
    return evidence_df

In [4]:
## https://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_nmf_lda.html#sphx-glr-auto-examples-applications-plot-topics-extraction-with-nmf-lda-py
def print_top_words(H, vocab, n_top_words, sample_topics):
    for topic_idx, topic in enumerate(H): ##(iterate thru each topic)
        ##this gives the indexes of the highest values in each topic word row 
        ##(which words are in the topic most often)
        top_features_ind = topic.argsort()[: -n_top_words - 1 : -1]
        ##in the vocab (aka feature_names) this is the actual list of the string of words
        top_features = [vocab[i] for i in top_features_ind]
        ## weights are whatever the value in the H matrix are - im not super sure of the meaning
        weights = topic[top_features_ind]
        ##gets each word  in the topic from matrix H
        if topic_idx in sample_topics:
            print(str(topic_idx) + str(top_features))
            print("\n")
        ##actually save the topics instead of printing them- and give them little numbers
        
        
def topics_per_doc(W, n_top_topics, samplearts, H, vocab, n_top_words, titles):
    for topic_idx, topic in enumerate(W): ##(iterate thru each document)
        ##this gives the indexes of the highest values in each topic word row 
        ##(which words are in the topic most often)
        top_features_ind = topic.argsort()[: -n_top_topics - 1 : -1]
        ##in the vocab (aka feature_names) this is the actual list of the string of words
        top_features = [i for i in top_features_ind]
        ## replace feature_names with the topics tnames that Im gonna make up
        ##top_features = [feature_names[i] for i in top_features_ind]
        ## weights are whatever the value in the H matrix are - im not super sure of the meaning
        weights = topic[top_features_ind]
        ##gets each word  in the topic from matrix H
        if topic_idx in samplearts:
            print("article: " + titles[topic_idx])
            ##get the article title from the titles list
            ## get the words from the words per topic function
            print_top_words(H, vocab, n_top_words, top_features)
            print("\n")


In [5]:
os.getcwd()
filename = 'C:/Users/julia/OneDrive/Desktop/NLP/climate-fever.jsonl'
climate_claims_df_INPUT = pd.read_json(filename, lines=True)
climate_claims_df = climate_claims_df_INPUT
## 1535 rows
climate_claims_df.head()

,claim_id,claim,claim_label,evidences
0,0,Global warming is driving polar bears toward extinction,SUPPORTS,"[{'evidence_id': 'Extinction risk from global warming:170', 'evidence_label': 'NOT_ENOUGH_INFO', 'article': 'Extinction risk from global warming',..."
1,5,"The sun has gone into ‘lockdown’ which could cause freezing weather, earthquakes and famine, say scientists",SUPPORTS,"[{'evidence_id': 'Famine:386', 'evidence_label': 'SUPPORTS', 'article': 'Famine', 'evidence': 'The current consensus of the scientific community i..."
2,6,The polar bear population has been growing.,REFUTES,"[{'evidence_id': 'Polar bear:1332', 'evidence_label': 'NOT_ENOUGH_INFO', 'article': 'Polar bear', 'evidence': '""Ask the experts: Are polar bear po..."
3,9,Ironic' study finds more CO2 has slightly cooled the planet,REFUTES,"[{'evidence_id': 'Atmosphere of Mars:131', 'evidence_label': 'NOT_ENOUGH_INFO', 'article': 'Atmosphere of Mars', 'evidence': 'CO2 in the mesospher..."
4,10,Human additions of CO2 are in the margin of error of current measurements and the gradual increase in CO2 is mainly from oceans degassing as the p...,REFUTES,"[{'evidence_id': 'Carbon dioxide in Earth's atmosphere:140', 'evidence_label': 'NOT_ENOUGH_INFO', 'article': 'Carbon dioxide in Earth's atmosphere..."


In [6]:
climate_claims_df.astype({'claim': 'string', 'claim_label':'category'}).dtypes
climate_claims_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1535 entries, 0 to 1534
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   claim_id     1535 non-null   int64 
 1   claim        1535 non-null   object
 2   claim_label  1535 non-null   object
 3   evidences    1535 non-null   object
dtypes: int64(1), object(3)
memory usage: 48.1+ KB


In [7]:
evidence_df = func_make_evidence_df(climate_claims_df, 'claim_id', 'evidences')
climate_claims_df = climate_claims_df.drop(columns=['evidences'])

In [8]:
evidence_df

,claim_id,evidence_id,evidence_label,article,evidence,entropy,votes
0,0,Extinction risk from global warming:170,NOT_ENOUGH_INFO,Extinction risk from global warming,"""Recent Research Shows Human Activity Driving Earth Towards Global Extinction Event"".",0.693147,"[SUPPORTS, NOT_ENOUGH_INFO, None, None, None]"
1,0,Global warming:14,SUPPORTS,Global warming,"Environmental impacts include the extinction or relocation of many species as their ecosystems change, most immediately the environments of coral ...",0.000000,"[SUPPORTS, SUPPORTS, None, None, None]"
2,0,Global warming:178,NOT_ENOUGH_INFO,Global warming,"Rising temperatures push bees to their physiological limits, and could cause the extinction of bee populations.",0.693147,"[SUPPORTS, NOT_ENOUGH_INFO, None, None, None]"
3,0,Habitat destruction:61,SUPPORTS,Habitat destruction,"Rising global temperatures, caused by the greenhouse effect, contribute to habitat destruction, endangering various species, such as the polar bear.",0.000000,"[SUPPORTS, SUPPORTS, None, None, None]"
4,0,Polar bear:1328,NOT_ENOUGH_INFO,Polar bear,"""Bear hunting caught in global warming debate"".",0.693147,"[SUPPORTS, NOT_ENOUGH_INFO, None, None, None]"
...,...,...,...,...,...,...,...
7670,3134,Bushfires in Australia:126,SUPPORTS,Bushfires in Australia,"Australia's climate has warmed by more than one degree Celsius over the past century, causing an increase in the frequency and intensity of heatwa...",0.000000,"[None, SUPPORTS, SUPPORTS, None, None]"
7671,3134,Effects of global warming:86,NOT_ENOUGH_INFO,Effects of global warming,"In the last 30–40 years, heat waves with high humidity have become more frequent and severe.",0.693147,"[None, SUPPORTS, NOT_ENOUGH_INFO, None, None]"
7672,3134,Global warming:155,NOT_ENOUGH_INFO,Global warming,"Many regions have probably already seen increases in warm spells and heat waves, and it is virtually certain that these changes will continue over...",0.693147,"[None, SUPPORTS, NOT_ENOUGH_INFO, None, None]"
7673,3134,Global warming:156,NOT_ENOUGH_INFO,Global warming,"Since the 1950s, droughts and heat waves have appeared simultaneously with increasing frequency.",0.693147,"[None, SUPPORTS, NOT_ENOUGH_INFO, None, None]"


In [9]:
climate_claims_df[climate_claims_df['claim_label'] == "SUPPORTS"]

,claim_id,claim,claim_label
0,0,Global warming is driving polar bears toward extinction,SUPPORTS
1,5,"The sun has gone into ‘lockdown’ which could cause freezing weather, earthquakes and famine, say scientists",SUPPORTS
5,11,They tell us that we are the primary forces controlling earth temperatures by the burning of fossil fuels and releasing their carbon dioxide.,SUPPORTS
6,14,The Great Barrier Reef is experiencing the most widespread bleaching ever recorded,SUPPORTS
11,28,"Volcanoes Melting West Antarctic Glaciers, Not Global Warming",SUPPORTS
...,...,...,...
1519,3104,"the satellite sensors show less warming in the lower troposphere (approximately 10,000 feet above the earth’s surface) than is reported by surface...",SUPPORTS
1526,3121,"""Cloud cover in models is poorly treated and inaccurately predicted.",SUPPORTS
1528,3123,"Given that a doubling of carbon dioxide would change the surface heat flux by only two watts per square meter, it is evident that a small change i...",SUPPORTS
1532,3130,"""There are other possible causes for climate change which could be associated with solar activity or related to variations in the temperature of t...",SUPPORTS


In [10]:
evidence_df[evidence_df['claim_id'] == 3134]

,claim_id,evidence_id,evidence_label,article,evidence,entropy,votes
7670,3134,Bushfires in Australia:126,SUPPORTS,Bushfires in Australia,"Australia's climate has warmed by more than one degree Celsius over the past century, causing an increase in the frequency and intensity of heatwa...",0.000000,"[None, SUPPORTS, SUPPORTS, None, None]"
7671,3134,Effects of global warming:86,NOT_ENOUGH_INFO,Effects of global warming,"In the last 30–40 years, heat waves with high humidity have become more frequent and severe.",0.693147,"[None, SUPPORTS, NOT_ENOUGH_INFO, None, None]"
7672,3134,Global warming:155,NOT_ENOUGH_INFO,Global warming,"Many regions have probably already seen increases in warm spells and heat waves, and it is virtually certain that these changes will continue over...",0.693147,"[None, SUPPORTS, NOT_ENOUGH_INFO, None, None]"
7673,3134,Global warming:156,NOT_ENOUGH_INFO,Global warming,"Since the 1950s, droughts and heat waves have appeared simultaneously with increasing frequency.",0.693147,"[None, SUPPORTS, NOT_ENOUGH_INFO, None, None]"
7674,3134,Heat wave:151,SUPPORTS,Heat wave,The effects of climate change have been projected to make heat waves in places such as Europe up to five times more likely to occur.,0.000000,"[None, SUPPORTS, SUPPORTS, None, None]"


In [11]:
evidence_df[evidence_df["entropy"] > 0.7]

,claim_id,evidence_id,evidence_label,article,evidence,entropy,votes
82,36,Global cooling:2,NOT_ENOUGH_INFO,Global cooling,The current scientific consensus on climate change is that the Earth underwent global warming throughout the 20th century and continues to warm.,1.098612,"[REFUTES, NOT_ENOUGH_INFO, SUPPORTS, None, None]"
112,55,Hockey stick controversy:175,SUPPORTS,Hockey stick controversy,"He said there had probably been no global warming since the 1940s, and ""Satellite data show no appreciable warming of the global atmosphere since ...",1.039721,"[REFUTES, NOT_ENOUGH_INFO, SUPPORTS, None, SUPPORTS]"
113,55,Hockey stick controversy:176,REFUTES,Hockey stick controversy,"In fact, if one ignores the unusual El Nino year of 1998, one sees a cooling trend.""",1.039721,"[REFUTES, NOT_ENOUGH_INFO, REFUTES, None, SUPPORTS]"
117,57,Medieval Warm Period:13,NOT_ENOUGH_INFO,Medieval Warm Period,"1200 AD, and was followed by a decline of temperature levels till between c. 1500 and c. 1700 the coldest phase since the last ice age occurred.""",1.098612,"[REFUTES, SUPPORTS, NOT_ENOUGH_INFO, None, None]"
122,60,Scientific consensus on climate change:136,REFUTES,Scientific consensus on climate change,Most of the climatic warming over the last 50 years is likely to have been caused by increased concentrations of greenhouse gases in the atmosphere.,1.039721,"[NOT_ENOUGH_INFO, REFUTES, REFUTES, None, SUPPORTS]"
...,...,...,...,...,...,...,...
7502,3062,Sea level rise:172,NOT_ENOUGH_INFO,Sea level rise,"In 2019, a study projected that in low emission scenario, sea level will rise 30 centimeters by 2050 and 69 centimetres by 2100, relatively to the...",1.098612,"[REFUTES, SUPPORTS, NOT_ENOUGH_INFO, None, None]"
7503,3062,Sea level rise:174,NOT_ENOUGH_INFO,Sea level rise,"There is the probability that the rise will be beyond 2 metres by 2100 in the high emission scenario, which will cause displacement of 187 million...",1.098612,"[NOT_ENOUGH_INFO, REFUTES, SUPPORTS, None, None]"
7530,3070,Poverty in the United Kingdom:108,NOT_ENOUGH_INFO,Poverty in the United Kingdom,"John McDonnell said the IFS analysis showed a “clear threat” to working people’s living standards, while the Liberal Democrats said that the “sava...",1.039721,"[SUPPORTS, NOT_ENOUGH_INFO, NOT_ENOUGH_INFO, None, REFUTES]"
7592,3103,Global warming controversy:375,NOT_ENOUGH_INFO,Global warming controversy,"It would be cheaper, they say, to wait for the impacts of climate change and then adapt to them"" says writer and environmental activist George Mon...",1.098612,"[SUPPORTS, REFUTES, NOT_ENOUGH_INFO, None, None]"


In [12]:
evidence_df['evidence']

0                                                                       "Recent Research Shows Human Activity Driving Earth Towards Global Extinction Event".
1       Environmental impacts include the extinction or relocation of many species as their ecosystems change, most immediately the environments of coral ...
2                                             Rising temperatures push bees to their physiological limits, and could cause the extinction of bee populations.
3        Rising global temperatures, caused by the greenhouse effect, contribute to habitat destruction, endangering various species, such as the polar bear.
4                                                                                                             "Bear hunting caught in global warming debate".
                                                                                ...                                                                          
7670    Australia's climate has warmed by more than 

In [13]:

climate_claims_df['claim']

0                                                                                                     Global warming is driving polar bears toward extinction
1                                                 The sun has gone into ‘lockdown’ which could cause freezing weather, earthquakes and famine, say scientists
2                                                                                                                 The polar bear population has been growing.
3                                                                                                 Ironic' study finds more CO2 has slightly cooled the planet
4       Human additions of CO2 are in the margin of error of current measurements and the gradual increase in CO2 is mainly from oceans degassing as the p...
                                                                                ...                                                                          
1530            About 60% of the warming observed fr

In [14]:
#climate_claims_df['claim'] = climate_claims_df['claim'].str.replace('carbon dioxide', 'carbon_dioxide')
#climate_claims_df['claim'] = climate_claims_df['claim'].str.replace('climate change', 'climate_change')
#climate_claims_df['claim'] = climate_claims_df['claim'].str.replace('global warming', 'global_warming')

In [15]:
vectorizer = CountVectorizer(min_df=0.05, max_df=0.7,ngram_range = (1,5), strip_accents = 'unicode',stop_words='english')
dtm = vectorizer.fit_transform(evidence_df['evidence'])
vocab = vectorizer.get_feature_names()
used_stop_words = vectorizer.get_stop_words()
## 35 topics
ldamodel = LDA(n_components=10)
## 
W = ldamodel.fit_transform(dtm) #(documents x topics)
H = ldamodel.components_ #(topics x words)
print(vocab)

C:\Users\julia\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['atmosphere', 'average', 'carbon', 'carbon dioxide', 'change', 'changes', 'climate', 'climate change', 'dioxide', 'earth', 'emissions', 'gas', 'gases', 'global', 'global warming', 'greenhouse', 'greenhouse gases', 'human', 'ice', 'increase', 'increased', 'level', 'ocean', 'rise', 'sea', 'sea level', 'surface', 'temperature', 'temperatures', 'warming', 'year', 'years']


In [16]:
#print(used_stop_words)

In [17]:
#PRsample = [1,2,3,4]
#PRsampID = climate_claims_df[climate_claims_df['claim_id'].isin(PRsample)].index.tolist()
#topics_per_doc(W, 3, PRsampID, H, vocab, 10, climate_claims_df['claim'].tolist())


print_top_words(H, vocab, 3, list(range(0, H.shape[0])))  



0['gases', 'years', 'greenhouse gases']


1['temperatures', 'year', 'increase']


2['climate', 'change', 'climate change']


3['ice', 'sea', 'years']


4['ocean', 'greenhouse', 'atmosphere']


5['temperature', 'earth', 'surface']


6['carbon', 'dioxide', 'carbon dioxide']


7['emissions', 'human', 'global']


8['rise', 'level', 'sea']


9['warming', 'global', 'global warming']




In [18]:
## sklearn DTM creation - document-term matrix
## info from https://stackoverflow.com/questions/39931941/create-a-dtm-from-large-corpus
## TF IDF is Term Frequency(TF) and Inverse Document Frequency(IDF)

## min df - a given word has to be in some % of articles (try 0.05)(originally I did 0.12)
## max df - Baird has set it lower than 0.90 in the past - couldbe 0.60 - 
## try printng out words that have doc freq of > 0.6 - are they informative? 
## start with n grams of 1 - and then add complexity 
## 
##vectorizer = CountVectorizer(min_df=0.05,max_df=0.95, ngram_range = (1,1), strip_accents = 'unicode', stop_words='english')
vectorizer = CountVectorizer(min_df=0.05, max_df=0.7,ngram_range = (1,5), strip_accents = 'unicode',stop_words='english')
dtm = vectorizer.fit_transform(climate_claims_df['claim'])
vocab = vectorizer.get_feature_names_out()
used_stop_words = vectorizer.get_stop_words()

In [19]:
#print(used_stop_words)

In [20]:
# topic model
## 35 topics
ldamodel = LDA(n_components=10)
## 
W = ldamodel.fit_transform(dtm) #(documents x topics)
H = ldamodel.components_ #(topics x words)


print(len(vocab))
#vocab

20


In [21]:
print(vocab)

['carbon' 'carbon dioxide' 'change' 'climate' 'climate change' 'co2'
 'dioxide' 'earth' 'emissions' 'global' 'global warming' 'ice' 'level'
 'rise' 'scientists' 'sea' 'temperature' 'temperatures' 'warming' 'years']


In [22]:
PRsample = [1,2,3,4]
PRsampID = climate_claims_df[climate_claims_df['claim_id'].isin(PRsample)].index.tolist()
topics_per_doc(W, 3, PRsampID, H, vocab, 10, climate_claims_df['claim'].tolist())


print_top_words(H, vocab, 3, list(range(0, H.shape[0])))  



0['sea', 'ice', 'years']


1['co2', 'emissions', 'years']


2['climate', 'change', 'climate change']


3['warming', 'climate', 'years']


4['ice', 'years', 'scientists']


5['carbon', 'dioxide', 'carbon dioxide']


6['global', 'warming', 'global warming']


7['level', 'rise', 'sea']


8['temperature', 'temperatures', 'global']


9['scientists', 'earth', 'years']


